In [ ]:
info = {}
info["ext"] = "mat"
#info["ext"] = "nii"
dir_path = "../RSNA-BTC-Datasets/train_"+info["ext"]
test_dir_path = "../RSNA-BTC-Datasets/test_"+info["ext"]
tumor_only_dir_path = "../RSNA-BTC-Datasets/ec_train_"+info["ext"]
tumor_only_test_dir_path = "../RSNA-BTC-Datasets/ec_test_"+info["ext"]
no_tumor_dir_path = "../RSNA-BTC-Datasets/no_tumor_train_"+info["ext"]
ext_test_1_dir_path = "../RSNA-BTC-Datasets/brats18_"+info["ext"]
ext_test_0_dir_path = "../RSNA-BTC-Datasets/OpenNeuroDS000221_ss_"+info["ext"]
new_dir_path = "../RSNA-BTC-Datasets/UPENN-GBM_"+info["ext"]

In [ ]:
import importlib
from utils import classifier_utils
from utils import dataset_utils
from utils.neural_networks import *
importlib.reload(classifier_utils)
importlib.reload(dataset_utils)

def predict_with_case_and_size(case, input_size):
    if case == "A":
        to = "m"#info["train_origin"]
        data_idx = 1
        if input_size == "3D":
            folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_14-52-37"
        else:
            folder = "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-59-16"
    elif case == "A_B":
        to = "n"
        data_idx = 2 #full
        if input_size == "3D":
            folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_14-52-37"
        else:
            folder = "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-59-16"
    elif case == "B":
        to = "n"
        data_idx = 1
        if input_size == "3D":
            folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_14-43-40"
        else:
            folder = "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-45-29"
    elif case == "B_A":
        to = "m"
        data_idx = 2 #full
        if input_size == "3D":
            folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_14-43-40"
        else:
            folder = "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-45-29"
    elif case == "AB":
        to = "mn"
        data_idx = 1
        if input_size == "3D":
            folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_15-38-50"
        else:
            folder = "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_16-20-40"

    #folder = "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_14-52-37"
    model_common_path = "../RSNA-BTC-Datasets/out_models/"
    model_folder = model_common_path + folder

    model_basefile, info = get_best_model(model_folder)
    m_modelfile = model_folder + "/" + model_basefile
    transform = None
    size = len(info["mri_types"])
    sel_slices = 1 if info["dims"] == 2 else None
    chosen_net = "alt" if info["dims"] == 2 else "sim"

    m_model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])

    #m_modelfile = m_modelfile_3d
    print(m_modelfile)
    print(info)

    print(f"Train origin: {to}")
    if to == "m":
        packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, m_path=dir_path) #m_path=dir_path, n_path=new_dir_path
    elif to == "n":
        packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, n_path=new_dir_path)
    elif to == "mn":
        packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, m_path=dir_path, n_path=new_dir_path)
    print(packs)
    loader_packs = dataset_utils.get_loaders(packs, info, is_fold=True, fold_num=1)
    print(loader_packs)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info("Using CUDA...")

    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print(loader_packs['KLF']['m_dataloaders'])
    selected_type = info["mri_types"][0]
    print(selected_type)
    dataloader = loader_packs[selected_type][f"{to}_dataloaders"]#m_dataloaders[0]
    val_loader = dataloader[data_idx]
    #val_loader = dataloader[2] #full

    print("\nPREDICTIONS:")
    print(folder)
    #print(f"Test set: {to} (B)")
    test_loader = val_loader
    m_test_pred, ids, X, y, y_pred, y_prob = classifier_utils.predict(m_model, device, m_modelfile, test_loader, size)
    #X_first = X[0].cpu().detach().numpy()
    #plt.imshow(X_first[0,96,:,:], cmap=cm.Greys_r)
    #plt.show()
    return m_test_pred, ids, X, y, y_pred, y_prob, m_model


In [ ]:
from captum.attr import IntegratedGradients
from matplotlib.colors import LinearSegmentedColormap
from captum.attr import GuidedGradCam, LayerGradCam, LayerAttribution
from captum.attr import visualization as viz
from captum.attr import Occlusion
import matplotlib.pyplot as plt

def get_xai(case, input_size, m_test_pred, ids, X, y, y_pred, y_prob, m_model):
    default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                       [(0, '#ffffff'),
                                                        (0.25, '#000000'),
                                                        (1, '#000000')], N=256)
    integrated_gradients = IntegratedGradients(m_model)
    occlusion = Occlusion(m_model)

    if input_size == "3D":
        if case == "A":
            sel_idx = {
                #"00532": 80,
                #"00206": 83,
                #"00389": 101
                "00491": 82,
                "00469": 124,
                "00764": 147
            }
        elif case == "A_B":
            sel_idx = {
                "UPENN-GBM-00302_11": 134,
                "UPENN-GBM-00604_11": 95,
                "UPENN-GBM-00149_11": 94
            }
        elif case == "B":
            sel_idx = {
                "UPENN-GBM-00128_11": 74,
                "UPENN-GBM-00410_11": 97,
                "UPENN-GBM-00381_11": 93
            }
        elif case == "B_A":
            sel_idx = {
                "00062": 79,
                "00090": 128,
                "00444": 100
            }
        elif case == "AB":
            sel_idx = {
                "00062": 79,
                "00269": 107,
                "00684": 96
            }
    else:
        if case == "A":
            sel_idx = {
                "00604": 0,
                "00684": 0,
                "00601": 0
            }
        elif case == "A_B":
            sel_idx = {
                "UPENN-GBM-00474_11": 0,
                "UPENN-GBM-00482_11": 0,
                "UPENN-GBM-00454_11": 0
            }
        elif case == "B":
            sel_idx = {
                "UPENN-GBM-00128_11": 0,
                "UPENN-GBM-00448_11": 0,
                "UPENN-GBM-00617_11": 0
            }
        elif case == "B_A":
            sel_idx = {
                "00144": 0,
                "00269": 0,
                "00800": 0
            }
        elif case == "AB":
            sel_idx = {
                "00470": 0,
                "00687": 0,
                "00251": 0
            }

    for i in range(len(ids)):
        idx = list(ids)[i]
        if idx in sel_idx.keys():
            print("Idx: "+idx)
            pred = y_pred[i]
            real = y[i]
            prob = y_prob[i][0]
            #tumor = tumor_y[i]

            print("Methylation pred value: "+str(pred))
            print("Methylation real value: "+str(real))
            print("Methylation prob value: "+str(prob))
            #print("Tumor real value: "+str(tumor))
            if input_size == "2D":
                img_test = X[i].unsqueeze(0)
            else:
                img_test = X[i].unsqueeze(0)#X[i][:,:,sel_idx[idx],:,:].unsqueeze(0)
                
            #print(np.shape(img_test.cpu().detach().numpy().transpose(2, 3, 4, 1, 0)))

            attributions_ig = integrated_gradients.attribute(img_test, target=0, n_steps=10)
            if input_size == "3D":
                attributions_occ = occlusion.attribute(img_test,
                                                     strides=(1, 8, 8, 8),
                                                     target=0,
                                                     sliding_window_shapes=(1, 8, 8, 8), #1,15,15,15
                                                     baselines=0)
            else:
                attributions_occ = occlusion.attribute(img_test,
                                                     strides=(1, 8, 8),
                                                     target=0,
                                                     sliding_window_shapes=(1, 8, 8), #1,15,15,15
                                                     baselines=0)
            if input_size == "2D":
                fig, axs = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze(0).cpu().detach().numpy(), (1, 2, 0)), #1,2,0
                                       img_test.cpu().detach().numpy().transpose(2, 3, 1, 0)[:, :, :, 0], #2,3,1,0
                                       method='heat_map',
                                       cmap=default_cmap,
                                       show_colorbar=False,
                                       sign='positive',
                                       outlier_perc=1)
                #fig.savefig(f"captum_results/XAI_3D/ig_{idx}_pred{pred}_real{real}_tumor{tumor}_prob{prob}.png")
                fig.savefig(f"captum_results/XAI_{input_size}/{case}/ig_{idx}_pred{pred}_real{real}_prob{prob}.png")
                try:
                    fig, axs = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze(0).cpu().detach().numpy(), (1, 2, 0)),
                                            img_test.cpu().detach().numpy().transpose(2, 3, 1, 0)[:, :, :, 0],
                                            ["original_image", "heat_map"],
                                            ["all", "positive"],
                                            show_colorbar=False,
                                            outlier_perc=2,
                                            )
                    #fig.savefig(f"captum_results/XAI_3D/occ_{idx}_pred{pred}_real{real}_tumor{tumor}_prob{prob}.png")
                    fig.savefig(f"captum_results/XAI_{input_size}/{case}/occ_{idx}_pred{pred}_real{real}_prob{prob}.png")
                except: 
                    print("Error in saving the occlusions")
                    continue
            else:
                fig, axs = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze(0).cpu().detach().numpy(), (2, 3, 1, 0)), #1,2,0
                                       img_test.cpu().detach().numpy().transpose(2, 3, 4, 1, 0)[sel_idx[idx], :, :, :, 0], #2,3,1,0
                                       method='heat_map',
                                       cmap=default_cmap,
                                       show_colorbar=False,
                                       sign='positive',
                                       outlier_perc=1)
                fig.savefig(f"captum_results/XAI_{input_size}/{case}/ig_{idx}_pred{pred}_real{real}_prob{prob}.png")
                try:
                    fig, axs = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze(0).cpu().detach().numpy(), (2, 3, 1, 0)),
                                            img_test.cpu().detach().numpy().transpose(2, 3, 4, 1, 0)[sel_idx[idx], :, :, :, 0],
                                            ["original_image", "heat_map"],
                                            ["all", "positive"],
                                            show_colorbar=False,
                                            outlier_perc=2,
                                            )
                    #fig.savefig(f"captum_results/XAI_3D/occ_{idx}_pred{pred}_real{real}_tumor{tumor}_prob{prob}.png")
                    fig.savefig(f"captum_results/XAI_{input_size}/{case}/occ_{idx}_pred{pred}_real{real}_prob{prob}.png")
                except:
                    print("Error in saving the occlusions")
                    continue
            #else:


In [ ]:
cases = ["AB"]#["A", "A_B", "B", "B_A", "AB"]
input_sizes = ["2D"] #["2D", "3D"]

for input_size in input_sizes:
    for case in cases:
        print(f"\t{input_size} {case}")
        m_test_pred, ids, X, y, y_pred, y_prob, m_model = predict_with_case_and_size(case, input_size)
        get_xai(case, input_size, m_test_pred, ids, X, y, y_pred, y_prob, m_model)